In [1]:
import numpy as np
import pandas as pd

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
import os 

DATA_PATH = os.getcwd()
print(DATA_PATH)

c:\NLP_Papers_Review\01_numpy&pandas


In [6]:
df = pd.read_csv(f"{DATA_PATH}\\data\\store_train_transactions.csv")

df.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523105 entries, 0 to 523104
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      523105 non-null  object
 1   구매일시    523105 non-null  object
 2   지점코드    523105 non-null  object
 3   대분류     523105 non-null  object
 4   중분류     523105 non-null  object
 5   브랜드코드   523105 non-null  int64 
 6   구매가격    523105 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 27.9+ MB


- 날짜 형식으로 변환

In [9]:
df["구매일시"] = pd.to_datetime(df["구매일시"])

- 날짜끼리 연산이 가능하다.

In [10]:
time_delta = df["구매일시"].max() - df["구매일시"].min()
time_delta

Timedelta('363 days 10:23:00')

In [11]:
time_delta.days

363

In [12]:
time_delta.total_seconds()

31400580.0

In [13]:
df["구매일시"].dt.weekday

0         5
1         5
2         5
3         5
4         5
         ..
523100    4
523101    4
523102    4
523103    4
523104    4
Name: 구매일시, Length: 523105, dtype: int32

In [16]:
df["구매일시"].dt.weekday.value_counts()

구매일시
4    100897
3     96614
2     80179
6     70327
1     69368
0     65879
5     39841
Name: count, dtype: int64

# Feature Engineering
- 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임을 만든다

In [17]:
features = pd.DataFrame(df["ID"].unique(),columns=["ID"])
features.head()

,ID
0,train_13219
1,train_5590
2,train_7200
3,train_3010
4,train_10851


## 구매일자를 이용한 특성생성

In [18]:
agg_list = [
        # 컬럼명, 집계 방식 튜플
        ('내점일수',lambda x : x.dt.date.nunique() ),
        ('구매주기',lambda x : ( x.max() - x.min() ).days / x.dt.date.nunique() ),
        ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4)),
        ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
        ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
        ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
        ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
        ('주구매요일', lambda x: x.dt.dayofweek.value_counts().index[0]),
        ('거래개월수', lambda x: x.dt.month.nunique() ),
    ]

tmp = df.groupby('ID')["구매일시"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12
1,train_5590,39,8.948718,0.073171,0.268293,0.341463,0.243902,0.146341,4,12
2,train_7200,17,21.117647,0.200000,0.509091,0.236364,0.109091,0.145455,0,8
3,train_3010,57,6.333333,0.258065,0.129032,0.282258,0.298387,0.290323,2,12
4,train_10851,14,23.142857,0.348837,0.093023,0.581395,0.162791,0.162791,2,9


In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

# tqdm 설정
tqdm.pandas()  # 일반적인 tqdm 설정 (콘솔 환경)
# tqdm_notebook.pandas()  # Jupyter Notebook에서 사용할 경우

agg_list = [
    ('내점일수', lambda x: x.dt.date.nunique()),
    ('구매주기', lambda x: (x.max() - x.min()).days / x.dt.date.nunique()),
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek > 4)),
    ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3, 4, 5]))),
    ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6, 7, 8]))),
    ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9, 10, 11]))),
    ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1, 2, 12]))),
    ('주구매요일', lambda x: x.dt.dayofweek.value_counts().index[0]),
    ('거래개월수', lambda x: x.dt.month.nunique()),
]

# tqdm을 사용한 그룹바이 연산
tmp = df.groupby('ID')["구매일시"].progress_apply(
    lambda x: pd.Series({name: func(x) for name, func in agg_list})
).reset_index()

features = features.merge(tmp, how='left')

features.head()


100%|██████████| 14940/14940 [00:17<00:00, 847.11it/s]


,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,level_1,구매일시
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,내점일수,46.000000
1,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,구매주기,7.413043
2,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,주말방문비율,0.186275
3,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,봄_구매비율,0.196078
4,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,여름_구매비율,0.235294


## 지점을 이용한 특성생성

In [20]:
agg_list = [
          ("방문지점수","nunique"),
          ('주구매지점', lambda x: x.mode()[0]),
    ]

tmp = df.groupby('ID')["지점코드"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,level_1,구매일시,방문지점수,주구매지점
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,내점일수,46.000000,1,A144000
1,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,구매주기,7.413043,1,A144000
2,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,주말방문비율,0.186275,1,A144000
3,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,봄_구매비율,0.196078,1,A144000
4,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,여름_구매비율,0.235294,1,A144000


# 브랜드코드를 이용한 특성생성

In [22]:
agg_list = [
             ('브랜드코드_nunique', 'nunique'),
             ]

tmp = df.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,level_1,구매일시,방문지점수,주구매지점,브랜드코드_nunique
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,내점일수,46.000000,1,A144000,45
1,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,구매주기,7.413043,1,A144000,45
2,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,주말방문비율,0.186275,1,A144000,45
3,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,봄_구매비율,0.196078,1,A144000,45
4,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,여름_구매비율,0.235294,1,A144000,45


## 중분류를 이용한 특성생성

In [23]:
agg_list = [
            ('중분류_nunique', 'nunique'),
            ('주구매_중분류', lambda x: x.mode()[0]),
            ]

tmp = df.groupby('ID')["중분류"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,level_1,구매일시,방문지점수,주구매지점,브랜드코드_nunique,중분류_nunique,주구매_중분류
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,내점일수,46.000000,1,A144000,45,35,용기보증
1,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,구매주기,7.413043,1,A144000,45,35,용기보증
2,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,주말방문비율,0.186275,1,A144000,45,35,용기보증
3,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,봄_구매비율,0.196078,1,A144000,45,35,용기보증
4,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,여름_구매비율,0.235294,1,A144000,45,35,용기보증


## 대분류를 이용한 특성생성

In [24]:
agg_list = [
            ('대분류_nunique', 'nunique'),
            ('주구매_대분류', lambda x: x.mode()[0]),
            ]

tmp = df.groupby('ID')["대분류"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,level_1,구매일시,방문지점수,주구매지점,브랜드코드_nunique,중분류_nunique,주구매_중분류,대분류_nunique,주구매_대분류
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,내점일수,46.000000,1,A144000,45,35,용기보증,7,케주얼_구두_아동
1,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,구매주기,7.413043,1,A144000,45,35,용기보증,7,케주얼_구두_아동
2,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,주말방문비율,0.186275,1,A144000,45,35,용기보증,7,케주얼_구두_아동
3,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,봄_구매비율,0.196078,1,A144000,45,35,용기보증,7,케주얼_구두_아동
4,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,여름_구매비율,0.235294,1,A144000,45,35,용기보증,7,케주얼_구두_아동


## 구매가격을 이용한 특성생성

In [25]:
agg_list = [
        ('총구매액',lambda x: x[x > 0].sum() ),
        ('구매건수', lambda x: x[x > 0].count() ),
        ('평균구매액', lambda x: x[x > 0].mean()),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ]

tmp = df.groupby(['ID'])["구매가격"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,...,대분류_nunique,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,7,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974
1,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,7,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974
2,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,7,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974
3,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,7,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974
4,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,7,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974


In [26]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             134460 non-null  object 
 1   내점일수           134460 non-null  int64  
 2   구매주기           134460 non-null  float64
 3   주말방문비율         134460 non-null  float64
 4   봄_구매비율         134460 non-null  float64
 5   여름_구매비율        134460 non-null  float64
 6   가을_구매비율        134460 non-null  float64
 7   겨울_구매비율        134460 non-null  float64
 8   주구매요일          134460 non-null  int32  
 9   거래개월수          134460 non-null  int64  
 10  level_1        134460 non-null  object 
 11  구매일시           134460 non-null  float64
 12  방문지점수          134460 non-null  int64  
 13  주구매지점          134460 non-null  object 
 14  브랜드코드_nunique  134460 non-null  int64  
 15  중분류_nunique    134460 non-null  int64  
 16  주구매_중분류        134460 non-null  object 
 17  대분류_nunique    134460 non-nul

```
고객이 기혼인 미혼인지 맞추는 데이터셋!!
결측치 처리는 신경 쓰지 말고 피처를 추가로 만들것

```

- 고객별 구매 증감률

In [27]:
# 고객별 구매 증감률
tmp = df.sort_values(by=['ID', '구매일시'])
tmp = tmp.groupby('ID')['구매가격'].agg(lambda x : x.pct_change().mean() ).reset_index()
tmp = tmp.rename(columns = {"구매가격": "구매가격_평균증감률"})
features = features.merge(tmp, how='left', on='ID')
features.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,거래개월수,...,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,구매가격_평균증감률
0,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974,0.750018
1,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974,0.750018
2,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974,0.750018
3,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974,0.750018
4,train_13219,46,7.413043,0.186275,0.196078,0.235294,0.205882,0.362745,4,12,...,케주얼_구두_아동,8428677,100,84286.77,278000,12000,-167000,2,65633.922974,0.750018


- 가정용품구매율

In [28]:
tmp = df.groupby('ID')['대분류'].agg(lambda x : x.isin(['가정용품파트', '가정용품', '생식품파트', '생식품']).mean() ).reset_index()
tmp = tmp.rename(columns = {"대분류": "가정용품구매율"})

features = features.merge(tmp, how='left', on='ID')
features.shape

(134460, 29)

- 고가상품 구매 비율

In [29]:
#고가상품 구매 비율
#quantile 0.7 정도로 기준을 잡고
#전체에서 quantile 0.7 정도 이상의 금액을 지불할 건에 대해서 count(normalize=True)

q7 = df["구매가격"].quantile(0.7)
agg_list = [
   ('고가상품구매비율' , lambda x : (x > q7).mean() )
]

tmp = df.groupby('ID')['구매가격'].agg(agg_list).reset_index()

features = features.merge(tmp, how = 'left')

features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 30 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             134460 non-null  object 
 1   내점일수           134460 non-null  int64  
 2   구매주기           134460 non-null  float64
 3   주말방문비율         134460 non-null  float64
 4   봄_구매비율         134460 non-null  float64
 5   여름_구매비율        134460 non-null  float64
 6   가을_구매비율        134460 non-null  float64
 7   겨울_구매비율        134460 non-null  float64
 8   주구매요일          134460 non-null  int32  
 9   거래개월수          134460 non-null  int64  
 10  level_1        134460 non-null  object 
 11  구매일시           134460 non-null  float64
 12  방문지점수          134460 non-null  int64  
 13  주구매지점          134460 non-null  object 
 14  브랜드코드_nunique  134460 non-null  int64  
 15  중분류_nunique    134460 non-null  int64  
 16  주구매_중분류        134460 non-null  object 
 17  대분류_nunique    134460 non-nul

In [30]:
#중분류에서 아동복 관련 구매를 한 사람 비율(아동복,아동,아동잡화,신생아,유아용품,유아복)이것들을
# 따로 리스트에 담아서 한 고객 당 구매비율 파악하기
agg_list = [

     ('아동물품구매' , lambda x : np.mean(x.isin(['아동복','아동','아동잡화','신생아','유아용품','유아복'])))

]
tmp = df.groupby('ID')['중분류'].agg(agg_list).reset_index()

features = features.merge(tmp, how = 'left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             134460 non-null  object 
 1   내점일수           134460 non-null  int64  
 2   구매주기           134460 non-null  float64
 3   주말방문비율         134460 non-null  float64
 4   봄_구매비율         134460 non-null  float64
 5   여름_구매비율        134460 non-null  float64
 6   가을_구매비율        134460 non-null  float64
 7   겨울_구매비율        134460 non-null  float64
 8   주구매요일          134460 non-null  int32  
 9   거래개월수          134460 non-null  int64  
 10  level_1        134460 non-null  object 
 11  구매일시           134460 non-null  float64
 12  방문지점수          134460 non-null  int64  
 13  주구매지점          134460 non-null  object 
 14  브랜드코드_nunique  134460 non-null  int64  
 15  중분류_nunique    134460 non-null  int64  
 16  주구매_중분류        134460 non-null  object 
 17  대분류_nunique    134460 non-nul

In [31]:
#구매일자의 시간을 이용해서 오후시간대 방문비율(오후1시~오후5시) / 저녁시간대 오후6시~10시 / 오전:9시~11시
#당연히 조건에 해당되는 것의 sum은 말이 안되고 방문을 한 횟수가 정확하지

agg_list = [
    ('오전_매장방문비율' , lambda x : np.mean((x.dt.hour >= 9) & (x.dt.hour <= 11))),
    ('오후_매장방문비율' , lambda x : np.mean((x.dt.hour >= 13) & (x.dt.hour <= 17))),
    ('저녁_밤_매장방문비율' , lambda x : np.mean(x.dt.hour >= 18))
]


tmp = df.groupby('ID')['구매일시'].agg(agg_list).reset_index()
features = features.merge(tmp, how = 'left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             134460 non-null  object 
 1   내점일수           134460 non-null  int64  
 2   구매주기           134460 non-null  float64
 3   주말방문비율         134460 non-null  float64
 4   봄_구매비율         134460 non-null  float64
 5   여름_구매비율        134460 non-null  float64
 6   가을_구매비율        134460 non-null  float64
 7   겨울_구매비율        134460 non-null  float64
 8   주구매요일          134460 non-null  int32  
 9   거래개월수          134460 non-null  int64  
 10  level_1        134460 non-null  object 
 11  구매일시           134460 non-null  float64
 12  방문지점수          134460 non-null  int64  
 13  주구매지점          134460 non-null  object 
 14  브랜드코드_nunique  134460 non-null  int64  
 15  중분류_nunique    134460 non-null  int64  
 16  주구매_중분류        134460 non-null  object 
 17  대분류_nunique    134460 non-nul

In [32]:
#평일 방문 횟수

agg_list = [
    ( '평일방문횟수' , lambda x : (x.dt.dayofweek < 5).sum() )
]


tmp = df.groupby('ID')['구매일시'].agg(agg_list).reset_index()
features = features.merge(tmp, how = 'left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 35 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             134460 non-null  object 
 1   내점일수           134460 non-null  int64  
 2   구매주기           134460 non-null  float64
 3   주말방문비율         134460 non-null  float64
 4   봄_구매비율         134460 non-null  float64
 5   여름_구매비율        134460 non-null  float64
 6   가을_구매비율        134460 non-null  float64
 7   겨울_구매비율        134460 non-null  float64
 8   주구매요일          134460 non-null  int32  
 9   거래개월수          134460 non-null  int64  
 10  level_1        134460 non-null  object 
 11  구매일시           134460 non-null  float64
 12  방문지점수          134460 non-null  int64  
 13  주구매지점          134460 non-null  object 
 14  브랜드코드_nunique  134460 non-null  int64  
 15  중분류_nunique    134460 non-null  int64  
 16  주구매_중분류        134460 non-null  object 
 17  대분류_nunique    134460 non-nul

- 시간대별 구매횟수 및 비율

In [33]:
agg_list= [
    ('아침구매횟수',lambda x : (x.dt.hour < 12).sum() ),
    ('오후구매횟수',lambda x : ( (x.dt.hour >= 12)&(x.dt.hour < 18) ).sum()),
    ('저녁구매횟수',lambda x : (x.dt.hour >= 18).sum()),
    ('아침구매비율',lambda x : (x.dt.hour < 12).mean() ),
    ('오후구매비율',lambda x : ( (x.dt.hour >= 12)&(x.dt.hour < 18) ).mean() ),
    ('저녁구매비율',lambda x : (x.dt.hour >= 18).mean() )
]
tmp = df.groupby('ID')["구매일시"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             134460 non-null  object 
 1   내점일수           134460 non-null  int64  
 2   구매주기           134460 non-null  float64
 3   주말방문비율         134460 non-null  float64
 4   봄_구매비율         134460 non-null  float64
 5   여름_구매비율        134460 non-null  float64
 6   가을_구매비율        134460 non-null  float64
 7   겨울_구매비율        134460 non-null  float64
 8   주구매요일          134460 non-null  int32  
 9   거래개월수          134460 non-null  int64  
 10  level_1        134460 non-null  object 
 11  구매일시           134460 non-null  float64
 12  방문지점수          134460 non-null  int64  
 13  주구매지점          134460 non-null  object 
 14  브랜드코드_nunique  134460 non-null  int64  
 15  중분류_nunique    134460 non-null  int64  
 16  주구매_중분류        134460 non-null  object 
 17  대분류_nunique    134460 non-nul

In [34]:
prod_list1 = ['생식품','생식품파트','공산품파트','공산품','잡화파트','가정용품파트','가정용품']
prod_list2 = ['아동_스포츠','아동_문화','케주얼_구두_아동','아동']
prod_list3 = ['여성캐쥬얼','패션잡화','여성정장','여성의류파트','여성캐주얼']
prod_list4 = ['남성정장스포츠','남성의류']
prod_list5 = ['스포츠캐쥬얼','영어덜트캐쥬얼','골프_유니캐쥬얼','잡화']
prod_list6 = ['로얄부띠끄','로얄부틱','명품잡화']
prod_list7 = ['영라이브','영플라자']

agg_list = [
    ('생활용품비율',lambda x : x.isin(prod_list1).mean() ),
    ('아동용품비율',lambda x : x.isin(prod_list2).mean() ),
    ('여성용품비율',lambda x : x.isin(prod_list3).mean() ),
    ('남성용품비율',lambda x : x.isin(prod_list4).mean() ),
    ('남녀공용용품비율',lambda x : x.isin(prod_list5).mean() ),
    ('명품비율',lambda x : x.isin(prod_list6).mean() ),
    ('기타비율',lambda x : x.isin(prod_list7).mean() )
]
tmp = df.groupby('ID')["대분류"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 48 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             134460 non-null  object 
 1   내점일수           134460 non-null  int64  
 2   구매주기           134460 non-null  float64
 3   주말방문비율         134460 non-null  float64
 4   봄_구매비율         134460 non-null  float64
 5   여름_구매비율        134460 non-null  float64
 6   가을_구매비율        134460 non-null  float64
 7   겨울_구매비율        134460 non-null  float64
 8   주구매요일          134460 non-null  int32  
 9   거래개월수          134460 non-null  int64  
 10  level_1        134460 non-null  object 
 11  구매일시           134460 non-null  float64
 12  방문지점수          134460 non-null  int64  
 13  주구매지점          134460 non-null  object 
 14  브랜드코드_nunique  134460 non-null  int64  
 15  중분류_nunique    134460 non-null  int64  
 16  주구매_중분류        134460 non-null  object 
 17  대분류_nunique    134460 non-nul

In [35]:
q1, q3 = df["구매가격"].quantile([0.25,0.75])
iqr = q3 - q1
w = 1.5
box_min = q1 - iqr * w
box_max = q3 + iqr * w
box_min, box_max

(-123250.0, 265470.0)

In [36]:
agg_list = [
    ('구매금액의 중간값',lambda x : x[x>0].median()),
    ('구매금액의 0.25 분위수',lambda x : x[x>0].quantile(0.25)),
    ('구매금액의 0.75 분위수',lambda x : x[x>0].quantile(0.75)),
    ('이상치_건수', lambda x:  (x > box_max).sum() )
    ]

tmp = df.groupby('ID')["구매가격"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 52 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              134460 non-null  object 
 1   내점일수            134460 non-null  int64  
 2   구매주기            134460 non-null  float64
 3   주말방문비율          134460 non-null  float64
 4   봄_구매비율          134460 non-null  float64
 5   여름_구매비율         134460 non-null  float64
 6   가을_구매비율         134460 non-null  float64
 7   겨울_구매비율         134460 non-null  float64
 8   주구매요일           134460 non-null  int32  
 9   거래개월수           134460 non-null  int64  
 10  level_1         134460 non-null  object 
 11  구매일시            134460 non-null  float64
 12  방문지점수           134460 non-null  int64  
 13  주구매지점           134460 non-null  object 
 14  브랜드코드_nunique   134460 non-null  int64  
 15  중분류_nunique     134460 non-null  int64  
 16  주구매_중분류         134460 non-null  object 
 17  대분류_nuniqu

In [37]:
agg_list = [
        # 컬럼명, 집계 방식 튜플
        ('많이구매한월',lambda x : x.dt.month.mode()[0])
         ]

tmp = df.groupby('ID')["구매일시"].agg(agg_list).reset_index()

features= features.merge(tmp, how='left' ,on = 'ID')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 53 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              134460 non-null  object 
 1   내점일수            134460 non-null  int64  
 2   구매주기            134460 non-null  float64
 3   주말방문비율          134460 non-null  float64
 4   봄_구매비율          134460 non-null  float64
 5   여름_구매비율         134460 non-null  float64
 6   가을_구매비율         134460 non-null  float64
 7   겨울_구매비율         134460 non-null  float64
 8   주구매요일           134460 non-null  int32  
 9   거래개월수           134460 non-null  int64  
 10  level_1         134460 non-null  object 
 11  구매일시            134460 non-null  float64
 12  방문지점수           134460 non-null  int64  
 13  주구매지점           134460 non-null  object 
 14  브랜드코드_nunique   134460 non-null  int64  
 15  중분류_nunique     134460 non-null  int64  
 16  주구매_중분류         134460 non-null  object 
 17  대분류_nuniqu

In [38]:
tmp = df.groupby('ID')["지점코드"].agg([
     ("주방문지점의구매빈도", lambda x : (x == x.mode()[0]).sum() )
     ]).reset_index()

features = features.merge(tmp, how='left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 54 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              134460 non-null  object 
 1   내점일수            134460 non-null  int64  
 2   구매주기            134460 non-null  float64
 3   주말방문비율          134460 non-null  float64
 4   봄_구매비율          134460 non-null  float64
 5   여름_구매비율         134460 non-null  float64
 6   가을_구매비율         134460 non-null  float64
 7   겨울_구매비율         134460 non-null  float64
 8   주구매요일           134460 non-null  int32  
 9   거래개월수           134460 non-null  int64  
 10  level_1         134460 non-null  object 
 11  구매일시            134460 non-null  float64
 12  방문지점수           134460 non-null  int64  
 13  주구매지점           134460 non-null  object 
 14  브랜드코드_nunique   134460 non-null  int64  
 15  중분류_nunique     134460 non-null  int64  
 16  주구매_중분류         134460 non-null  object 
 17  대분류_nuniqu

In [39]:
tmp = df.groupby('ID')['브랜드코드'].agg([('각고객별_선호브랜드코드',lambda x: x.mode()[0])]).reset_index()

features = features.merge(tmp, how='left', on ='ID')

features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 55 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              134460 non-null  object 
 1   내점일수            134460 non-null  int64  
 2   구매주기            134460 non-null  float64
 3   주말방문비율          134460 non-null  float64
 4   봄_구매비율          134460 non-null  float64
 5   여름_구매비율         134460 non-null  float64
 6   가을_구매비율         134460 non-null  float64
 7   겨울_구매비율         134460 non-null  float64
 8   주구매요일           134460 non-null  int32  
 9   거래개월수           134460 non-null  int64  
 10  level_1         134460 non-null  object 
 11  구매일시            134460 non-null  float64
 12  방문지점수           134460 non-null  int64  
 13  주구매지점           134460 non-null  object 
 14  브랜드코드_nunique   134460 non-null  int64  
 15  중분류_nunique     134460 non-null  int64  
 16  주구매_중분류         134460 non-null  object 
 17  대분류_nuniqu

In [40]:
agg_list = {
  '구매가격':[
      ('환불여부' , lambda x :1 if (x<0).any() else 0 ),
      ('평균 구매액 대비 최대 구매액 비율',lambda x : round(x.max()/x.mean(),2)),
      ('평균 구매액 대비 최소 구매액 비율',lambda x : round(x[x > 0].min()/x.mean(),2)),
      ('구매액 대비 환불 비율',lambda x : round(abs(x[x<0].sum())/x.sum(),2))
  ]
}

tmp = df.groupby('ID').agg(agg_list)
tmp.columns = tmp.columns.droplevel()
tmp = tmp.reset_index()

features = features.merge(tmp, how='left', on ='ID')

features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 59 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   134460 non-null  object 
 1   내점일수                 134460 non-null  int64  
 2   구매주기                 134460 non-null  float64
 3   주말방문비율               134460 non-null  float64
 4   봄_구매비율               134460 non-null  float64
 5   여름_구매비율              134460 non-null  float64
 6   가을_구매비율              134460 non-null  float64
 7   겨울_구매비율              134460 non-null  float64
 8   주구매요일                134460 non-null  int32  
 9   거래개월수                134460 non-null  int64  
 10  level_1              134460 non-null  object 
 11  구매일시                 134460 non-null  float64
 12  방문지점수                134460 non-null  int64  
 13  주구매지점                134460 non-null  object 
 14  브랜드코드_nunique        134460 non-null  int64  
 15  중분류_nunique      

가능한 features

- 주구매 대분류/중분류:
-- '아동', '유아', '주니어' 키워드가 들어간 사람 (기혼-유자녀)
-- '영' 키워드 들어간 사람 (미혼)
-- '여성' or '남성', '식품', '잡화' 키워드의 구매 비율 (특정 카테고리에 집중도가 높은지, 고루 구매하는지)


In [41]:
# 주구매 시간 - 구매시간의 hour만 따져서 최빈값

agg_list = [
    ("주_구매시간", lambda x: x.dt.hour.mode()[0])
    ]

tmp = df.groupby('ID')['구매일시'].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')

features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 60 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   134460 non-null  object 
 1   내점일수                 134460 non-null  int64  
 2   구매주기                 134460 non-null  float64
 3   주말방문비율               134460 non-null  float64
 4   봄_구매비율               134460 non-null  float64
 5   여름_구매비율              134460 non-null  float64
 6   가을_구매비율              134460 non-null  float64
 7   겨울_구매비율              134460 non-null  float64
 8   주구매요일                134460 non-null  int32  
 9   거래개월수                134460 non-null  int64  
 10  level_1              134460 non-null  object 
 11  구매일시                 134460 non-null  float64
 12  방문지점수                134460 non-null  int64  
 13  주구매지점                134460 non-null  object 
 14  브랜드코드_nunique        134460 non-null  int64  
 15  중분류_nunique      

In [42]:
tmp = df.groupby('ID')['대분류'].agg(lambda x: (x.str.contains("아동") | x.str.contains("유아")).mean() )
tmp = tmp.reset_index()
tmp = tmp.rename(columns = {"대분류": "아동관련상품구매비율"})
features = pd.merge(features,tmp, how="left", on="ID")
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 61 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   134460 non-null  object 
 1   내점일수                 134460 non-null  int64  
 2   구매주기                 134460 non-null  float64
 3   주말방문비율               134460 non-null  float64
 4   봄_구매비율               134460 non-null  float64
 5   여름_구매비율              134460 non-null  float64
 6   가을_구매비율              134460 non-null  float64
 7   겨울_구매비율              134460 non-null  float64
 8   주구매요일                134460 non-null  int32  
 9   거래개월수                134460 non-null  int64  
 10  level_1              134460 non-null  object 
 11  구매일시                 134460 non-null  float64
 12  방문지점수                134460 non-null  int64  
 13  주구매지점                134460 non-null  object 
 14  브랜드코드_nunique        134460 non-null  int64  
 15  중분류_nunique      

- 고객 ID별 최근 구매 이후 경과일수
- 최근 구매 이후 경과일수를 이용한 특성생성

In [43]:
tmp = df.sort_values(['ID', '구매일시'])

tmp['최근 구매 이후 경과일수'] = tmp.groupby('ID')['구매일시'].diff()

tmp['최근 구매 이후 경과일수'] = tmp['최근 구매 이후 경과일수'].dt.days

agg_list = [
    ('최근_구매_이후_경과일수_평균', 'mean'),
    ('최근_구매_이후_경과일수_최대', 'max'),
    ('최근_구매_이후_경과일수_최소', 'min'),
]

tmp = tmp.groupby('ID')['최근 구매 이후 경과일수'].agg(agg_list).reset_index()
features = features.merge(tmp, how='left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 64 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   134460 non-null  object 
 1   내점일수                 134460 non-null  int64  
 2   구매주기                 134460 non-null  float64
 3   주말방문비율               134460 non-null  float64
 4   봄_구매비율               134460 non-null  float64
 5   여름_구매비율              134460 non-null  float64
 6   가을_구매비율              134460 non-null  float64
 7   겨울_구매비율              134460 non-null  float64
 8   주구매요일                134460 non-null  int32  
 9   거래개월수                134460 non-null  int64  
 10  level_1              134460 non-null  object 
 11  구매일시                 134460 non-null  float64
 12  방문지점수                134460 non-null  int64  
 13  주구매지점                134460 non-null  object 
 14  브랜드코드_nunique        134460 non-null  int64  
 15  중분류_nunique      

구매시간대별 최고가 특성생성

In [44]:
df['구매시간대'] = df['구매일시'].dt.hour

tmp = df.groupby(['ID', '구매시간대'])['구매가격'].agg('max').reset_index()

agg_list = [
    ('시간대별_구매_최고가_평균', 'mean'),
    ('시간대별_구매_최고가_최대', 'max'),
    ('시간대별_구매_최고가_최소', 'min'),
]

tmp = tmp.groupby("ID")["구매가격"].agg(agg_list).reset_index()

features = features.merge(tmp, on='ID', how='left')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 67 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   134460 non-null  object 
 1   내점일수                 134460 non-null  int64  
 2   구매주기                 134460 non-null  float64
 3   주말방문비율               134460 non-null  float64
 4   봄_구매비율               134460 non-null  float64
 5   여름_구매비율              134460 non-null  float64
 6   가을_구매비율              134460 non-null  float64
 7   겨울_구매비율              134460 non-null  float64
 8   주구매요일                134460 non-null  int32  
 9   거래개월수                134460 non-null  int64  
 10  level_1              134460 non-null  object 
 11  구매일시                 134460 non-null  float64
 12  방문지점수                134460 non-null  int64  
 13  주구매지점                134460 non-null  object 
 14  브랜드코드_nunique        134460 non-null  int64  
 15  중분류_nunique      

- 접근 아이디어: 피처는 타겟에 맞는 핵심만 있으면 된다. 많을 필요 없음. 이진분류를 위한 데이터만 있으면 된다 생각
- 주 계절 구분한 이유>>특정 달에 결혼이 많음>>ex 5월의 신부,>>이때 매장을 많이 방문한다>>결혼했을 확률 up

```
분류별 인기상품 보여주기
브랜드의 신용도 순위 메기기
브랜드의 인기상품 보여주기
회원등급을 입력하기
```

-- 체류시간: 같은 날 와서 마지막 구매 시간 - 첫 구매 시간

In [45]:
df["구매날짜"] = df["구매일시"].dt.date
tmp = df.groupby(["ID","구매날짜"])["구매일시"].agg(lambda x : (x.max() - x.min()).total_seconds() ).reset_index()
tmp

,ID,구매날짜,구매일시
0,train_0,2004-05-07,0.0
1,train_0,2004-06-05,600.0
2,train_0,2004-06-14,0.0
3,train_0,2004-07-29,1200.0
4,train_0,2004-09-01,2220.0
...,...,...,...
271846,train_9999,2005-01-31,0.0
271847,train_9999,2005-02-18,4140.0
271848,train_9999,2005-02-23,7800.0
271849,train_9999,2005-04-01,1980.0


In [46]:
agg_list = [
    ("체류시간_max","max"),
    ("체류시간_min","min"),
    ("체류시간_std","std"),
    ("체류시간_mean","mean"),
    ("체류시간_sum","sum")
]
tmp2 = tmp.groupby("ID")["구매일시"].agg(agg_list).reset_index()
features = pd.merge(features, tmp2, how="left", on="ID")
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134460 entries, 0 to 134459
Data columns (total 72 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   134460 non-null  object 
 1   내점일수                 134460 non-null  int64  
 2   구매주기                 134460 non-null  float64
 3   주말방문비율               134460 non-null  float64
 4   봄_구매비율               134460 non-null  float64
 5   여름_구매비율              134460 non-null  float64
 6   가을_구매비율              134460 non-null  float64
 7   겨울_구매비율              134460 non-null  float64
 8   주구매요일                134460 non-null  int32  
 9   거래개월수                134460 non-null  int64  
 10  level_1              134460 non-null  object 
 11  구매일시                 134460 non-null  float64
 12  방문지점수                134460 non-null  int64  
 13  주구매지점                134460 non-null  object 
 14  브랜드코드_nunique        134460 non-null  int64  
 15  중분류_nunique      

- 저장하기

In [47]:
features.to_csv(f"{DATA_PATH}features.csv", index=False)